In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from subprocess import check_call
from glob import glob

import socket

import yaml
import textwrap

import cesm_tools

project = 'mom6-marbl-cases'
this_notebook = 'create-new-case.ipynb'

assert 'cheyenne' in socket.gethostname(), (
    'this notebook must be run on cheyenne login node'
)

In [3]:
remote = 'git@github.com:mnlevy1981/cesm.git'

coderoot = f"/glade/work/{os.environ['USER']}/codes"
cesmtag = 'cesm2_3_beta08+mom6_marbl'

cesmroot = cesm_tools.code_checkout(remote, coderoot, cesmtag)

In [4]:
USER = os.environ["USER"]
scriptroot = os.getcwd()

machine = "cheyenne"
if machine in ["cheyenne"]:
    inputdata = "/glade/p/cesmdata/cseg/inputdata"
    scratch = f"/glade/scratch/{USER}"
    project_root = f'/glade/work/{USER}/{project}'
    project_data = f'{project_root}/data'
    caseroot_root = f'{project_root}/cases'
else:
    raise ValueError(f"unknown machine: {machine}")

In [5]:
clobber = True

project = "P93300670"

member_id = 1
compset = 'C1850MOMMARBL'
resolution = "TL319_t061"

description = """
my first MOM6 run
"""

case = f'{compset.lower()}.{resolution.lower()}.{project}.{member_id:03d}'

# run setting
resubmit = 0
stop_n = 1
stop_option = "nmonth"
rest_n = 1
rest_option = "nyear"
wallclock = "12:00:00"
queue = "economy"

# paths
caseinfo_file = f"case-info/{case}.yml"
caseroot = f"{caseroot_root}/{case}"
rundir = f"{scratch}/{case}"
archive_root = f"{scratch}/archive/{case}"

In [6]:
# ensure root directories
os.makedirs(os.path.dirname(caseroot_root), exist_ok=True)
os.makedirs(os.path.dirname(caseinfo_file), exist_ok=True)

if clobber:
    check_call(['rm', '-f', caseinfo_file])

if os.path.exists(caseinfo_file):
    raise ValueError(f'case {case} exists\nset clobber=True to remove')


with open(caseinfo_file, "w") as fid:
    yaml.dump(
        dict(
            case=case,
            description=description,
            caseroot=caseroot,
            rundir=rundir,
            archive_root=archive_root,
            compset=compset,
            resolution=resolution,
        ),
        fid,
    )
if os.path.exists(caseroot):
    print(f"deleting {caseroot}")
    check_call(['rm', '-fr', caseroot])
    
if os.path.exists(rundir):
    print(f"deleting {rundir}")
    check_call(['rm', '-fr', rundir])

In [7]:
check_call(
    [
        "./create_newcase",
        "--case",
        caseroot,
        "--project",
        project,
        "--machine",
        machine,
        "--res",
        resolution,
        "--compset",
        compset,
        "--run-unsupport",
    ],
    cwd=f"{cesmroot}/cime/scripts",
)

In [8]:
def xmlchange(arg):
    """call xmlchange"""
    check_call(["./xmlchange", arg], cwd=caseroot)

In [9]:
# initialization
xmlchange("RUN_TYPE=startup")
xmlchange("OCN_BGC_HIST_VERT_GRID=native")

xmlchange(f"RESUBMIT={resubmit}")
xmlchange(f"STOP_OPTION={stop_option}")
xmlchange(f"STOP_N={stop_n}")
xmlchange(f"REST_OPTION={rest_option}")
xmlchange(f"REST_N={rest_n}")
xmlchange(f"JOB_WALLCLOCK_TIME={wallclock}")
xmlchange(f"JOB_QUEUE={queue}");

In [10]:
check_call(['./case.setup'], cwd=caseroot)

In [11]:
src_mods_files = {
    'mom': glob(f"{scriptroot}/SourceMods/src.mom/*"),
}
for component, files in src_mods_files.items():
    for src in files:
        src_basename = os.path.basename(src)
        dst = f"{caseroot}/SourceMods/src.{component}/{src_basename}"
        shutil.copyfile(src, dst)

In [12]:
def list_to_nml_list(lst):
    return ",".join([f"'{v}'" for v in lst])

user_nl = dict()

for key, nl in user_nl.items():
    with open(f"{caseroot}/user_nl_{key}", 'w') as fid:
        fid.write(user_nl[key])

In [13]:
check_call(['qcmd', '-A', PROJECT, '--', './case.build'], cwd=caseroot)

In [14]:
check_call(['qcmd', '-A', project, '--', './case.build'], cwd=caseroot)

In [15]:
check_call(['./preview_namelists'], cwd=caseroot)

In [16]:
check_call(['./case.submit'], cwd=caseroot)